In [ ]:
!pip install onnxruntime

In [ ]:
!git clone https://github.com/Debatrix/DFSNet.git

In [ ]:
from DFSNet.model.quality_model import MobileNetV2_Lite
import torchvision.transforms as transforms
from PIL import Image
import torch
import numpy as np
import cv2
import onnxruntime as ort
import matplotlib.pyplot as plt
import os
import csv
from tqdm import tqdm
from scipy.signal import convolve2d, find_peaks
from scipy.ndimage import gaussian_filter, gaussian_filter1d, map_coordinates
from scipy.optimize import minimize
from skimage.transform import radon
from skimage.draw import ellipse, disk
from scipy.fftpack import fft2, fftshift

In [ ]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

model_segment_path = "segment_model_path"

model_score_path_1 = "pretrained_model_DFSNet_path"
model_score_path_2 = "pretrained_model_DFSNet_path"

input_folder = "input_folder_path"
output_csv_path = "criteria_choosing_path"

In [ ]:
session = ort.InferenceSession(model_segment_path)

In [ ]:
def pixel_count_score(iris_mask): 
    Q_pixel_count = np.sum(iris_mask > 0)
    return float(Q_pixel_count/307200 * 100)


def sharpness_score(image):
    gaussianX = cv2.Sobel(image, cv2.CV_16U, 1, 0)
    gaussianY = cv2.Sobel(image, cv2.CV_16U, 0, 1)
    sharpness_indicator = np.mean(np.sqrt(gaussianX**2 + gaussianY**2))
    return float(sharpness_indicator)

def off_angle_score(light_spots, pupil_center, pupil_radius):
    distances = [np.linalg.norm(np.array(spot) - np.array(pupil_center)) for spot in light_spots]
    max_distance = max(distances, default=0)
    return float(max_distance / pupil_radius) if max_distance > pupil_radius else 0


def dilation_score(iris_param, pupil_param):
    iris_radius = iris_param[2]
    pupil_radius = pupil_param[2]
    all_area = np.pi*(iris_radius**2)
    usable_iris_area = np.pi * (iris_radius**2 - pupil_radius**2)
    if usable_iris_area > 0:
        usable_area_indicator = (usable_iris_area / all_area) * 100
    else:
        usable_area_indicator = 0.0
    return float(usable_area_indicator)


def gray_level_spread_score(img, mask):
    img = img.astype(int)
    img[mask != True] = -1
    usable_pix_num = np.sum(mask)
    if usable_pix_num == 0:
        return 0.0
    gray_level_spread_indicator = 0.0
    for i in range(256):
        p = np.sum(img == i) / usable_pix_num
        if p > 0:
            gray_level_spread_indicator -= p * np.log2(p)
    return float(gray_level_spread_indicator)


def segment_image(model, image_array):
    image = Image.fromarray(image_array).convert("RGB")
    
    resize = transforms.Resize((640, 480))
    image_resized = resize(image)
    
    transform = transforms.Compose([
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.480], std=[0.200], inplace=False)
    ])
    
    image_tensor = transform(image_resized).unsqueeze(0).to(device)
    print("Image tensor shape:", image_tensor.shape)

    model.eval()
    with torch.no_grad():
        pred = model(image_tensor)[0]

        return pred


model = MobileNetV2_Lite()
model.to(device)
checkpoint = torch.load(model_score_path_2, map_location=device, weights_only=True)
model.load_state_dict(checkpoint['model'])


def process_image_and_calculate_metrics(image):
    original_image = cv2.imread(image, cv2.IMREAD_GRAYSCALE)

    input_size = (640, 480)
    
    def preprocess_image(img):
        img = cv2.resize(img, input_size)
        img = img / 255.0
        mean = np.array([0.485, 0.456, 0.406])
        std = np.array([0.229, 0.224, 0.225])
        img = np.stack([img, img, img], axis=-1)
        img = (img - mean) / std
        img = np.transpose(img, (2, 0, 1))
        img = np.expand_dims(img, axis=0).astype(np.float32)
        return img
    
    input_tensor = preprocess_image(original_image)
    input_name = session.get_inputs()[0].name
    output_name = session.get_outputs()[0].name
    output = session.run([output_name], {input_name: input_tensor})[0]
    
    def postprocess_output(output):
        segmentation_maps = np.squeeze(output)
        binary_masks = (segmentation_maps > 0.5).astype(np.uint8)
        return binary_masks

    masks = postprocess_output(output)
    mask_iris = masks[1]
    mask_pupil = masks[2]
    mask_iris_resized = cv2.resize(mask_iris, (original_image.shape[1], original_image.shape[0]), interpolation=cv2.INTER_NEAREST)
    mask_pupil_resized = cv2.resize(mask_pupil, (original_image.shape[1], original_image.shape[0]), interpolation=cv2.INTER_NEAREST)
    
    def define_circle_param(mask):
        contours, _ = cv2.findContours(mask.astype(np.uint8), cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
        largest_contour = max(contours, key=cv2.contourArea)
        (x, y), radius = cv2.minEnclosingCircle(largest_contour)
        return x, y, radius
    
    def find_light_spots(img, threshold=200):
        _, thresholded = cv2.threshold(img, threshold, 255, cv2.THRESH_BINARY)
        return np.column_stack(np.where(thresholded == 255))
    
    iris_param = define_circle_param(mask_iris_resized)
    pupil_param = define_circle_param(mask_pupil_resized)
    pupil_radius = pupil_param[2]
    pupil_center = (pupil_param[0], pupil_param[1])
    light_spots = find_light_spots(original_image) 

    Q_pixel_value = round(pixel_count_score(mask_iris_resized), 2)
    sharpness_value = round(sharpness_score(original_image), 2)
    off_angle_value = round(off_angle_score(light_spots, pupil_center, pupil_radius), 2)
    dilation_value = round(dilation_score(iris_param, pupil_param), 2)
    gray_level_spread_value = round(gray_level_spread_score(original_image, mask_iris_resized), 2)
    quality_score = round(float(segment_image(model, original_image)), 2)

    metrics = {"Pixel Count Score": Q_pixel_value,
               "Sharpness Score": sharpness_value,
               "Off-Angle Score": off_angle_value,
               "Dilation Score": dilation_value,
               "GLS Score": gray_level_spread_value,
               "Quality Score": quality_score}
    return metrics

In [ ]:
import numpy as np
import cv2
import os
import csv
from tqdm import tqdm
from scipy.signal import convolve2d
from skimage.draw import disk
from skimage.transform import radon
from scipy.ndimage import gaussian_filter1d

def evaluate_folder(folder_path, csv_path):
    csv_headers = [
        "Image Name", "Pixel Count Score", "Sharpness Score", "Off-Angle Score", 
        "Dilation Score", "GLS Score", "Quality Score"
    ]

    def print_metrics_data(metrics):
        """In ra kiểu dữ liệu và giá trị của từng metric."""
        for key, value in metrics.items():
            print(f"Metric: {key}")
            print(f"  Type: {type(value)}")
            print(f"  Value: {value}\n")

    with open(csv_path, mode="w", newline="") as csvfile:
        writer = csv.writer(csvfile)
        writer.writerow(csv_headers)

        for image_name in tqdm(os.listdir(folder_path)):
            image_path = os.path.join(folder_path, image_name)
            if not image_path.lower().endswith(('.jpg', '.jpeg', '.png', '.bmp', '.tiff')):
                continue

            img = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)
            if img is None:
                print(f"Could not read image {image_name}, skipping.")
                continue

            try:
                metrics = process_image_and_calculate_metrics(image_path)

                # In ra kiểu dữ liệu và giá trị của từng metric
                print(f"Metrics for {image_name}:")
                print_metrics_data(metrics)

                # Ghi dữ liệu vào CSV
                row = [image_name] + [metrics.get(header, "N/A") for header in csv_headers[1:]]
                writer.writerow(row)

            except Exception as e:
                print(f"Error processing image {image_name}: {e}")

In [ ]:
evaluate_folder(input_folder, output_csv_path)

print(f"Evaluation completed. Results saved to {output_csv_path}.")